In [2]:
# first, test generative model on one dimensional graphs
import sys
sys.path.append('./HCM') # outdated due to abstraction learning implementation
import pandas as pd
import seaborn as sns
import numpy as np
from Learning import *
from CG1 import *

In [7]:
import pandas as pd
df = pd.read_csv('/Users/swu/Desktop/research/motif_learning/data/simonsays_ed/data.csv')

dfm = {}  # model dataframe
dfm['blockcollect'] = []
dfm['ID'] = []
dfm['condition'] = []
dfm['correctcollect'] = []
dfm['p'] = []
dfm['trialcollect'] = []
dfm['recall_likelihood'] = [] # the likelihood of subjects recalling the exact piece of sequence, given the representation learned so far

column_names = ['ID', 'condition', 'recallcollect', 'instructioncollect','trialcollect', 'correctcollect']
dfs = pd.DataFrame(columns=column_names)  # Note that there are now row data inserted.


seql = 12
len_train = 40
len_test = 8

def convert_sequence(seq,keyassignment):
    keyassignment = list(keyassignment)
    ka = [keyassignment[2],keyassignment[7],keyassignment[12],keyassignment[17],keyassignment[22],keyassignment[27]]
    proj_seq = [ka.index(item)+ 1 for item in seq]
    return proj_seq


def calculate_prob(chunk_record, cg):
    p = 1
    for key in list(chunk_record.keys()):# key is the encoding time
        p = p*cg.chunks[chunk_record[key][0][0]].count/np.sum([item.count for item in cg.chunks])
    return p

def convert_sequence_backward_to_key(seq, keyassignment):
    keyassignment = list(keyassignment)
    inst_seq = []
    ka = [keyassignment[2], keyassignment[7], keyassignment[12], keyassignment[17], keyassignment[22],
          keyassignment[27]]
    for i in range(0, 12):
        inst_seq.append(ka[int(seq[i,0,0])-1])
    return inst_seq

    

for sub in np.unique(list(df['ID'])):
    # initialize chunking part with specified parameters
    cg = CG1(DT=0.1, theta=0.996)
    for trial in range(1, len_train + 3*len_test+ 1):
        ins_seq = df[(df['ID'] == sub)].iloc[(trial-1)*seql:trial*seql, :][
            'instructioncollect']
        subj_recall =  df[(df['ID'] == sub)].iloc[(trial-1)*seql:trial*seql, :][
            'recallcollect']
        keyassignment = list(df[(df['ID'] == sub)].iloc[(trial-1)*seql:trial*seql, :][
            'keyassignment'])[0]
        condition = list(df[(df['ID'] == sub)].iloc[(trial-1)*seql:trial*seql, :][
            'condition'])[0]
        block = list(df[(df['ID'] == sub)].iloc[(trial-1)*seql:trial*seql, :][
            'blockcollect'])[0]
        
        proj_seq = convert_sequence(list(ins_seq),keyassignment)
        proj_seq = np.array(proj_seq).reshape([-1, 1, 1])
    
        cg, chunkrecord = hcm_learning(proj_seq, cg,chunk = False)  # with the rational chunk models, rational_chunk_all_info(seq, cg)
        recalled_seq,ps = recall(cg, firstitem=proj_seq[0,0,0])
        
        keypress_recalled_seq = convert_sequence_backward_to_key(recalled_seq,keyassignment)
        
        p_seq = np.prod(ps)# evaluate the probability of a sequence
        
        proj_recalled_seq = convert_sequence(list(subj_recall),keyassignment)
        proj_recalled_seq = np.array(proj_recalled_seq).reshape([-1, 1, 1])
        cg, chunkrecord = hcm_learning(proj_recalled_seq, cg, learn = False, chunk = False)  # with the rational chunk models, rational_chunk_all_info(seq, cg)
        recall_likelihood = calculate_prob(chunkrecord, cg)# evaluate the probability of a sequence
        
        dfm['blockcollect'].append(block)
        dfm['ID'].append(sub)
        dfm['condition'].append(condition)
        dfm['correctcollect'].append(acc_eval1d(recalled_seq, proj_seq))
        dfm['p'].append(p_seq)
        dfm['trialcollect'].append(trial)
        dfm['recall_likelihood'].append(recall_likelihood)
        
        ins_list = ins_seq.tolist()
        print(keypress_recalled_seq, ins_list)
        for i in range(0, 12):
            if keypress_recalled_seq[i] == ins_list[i]:
                correctcollect = 1
            else: correctcollect = 0
            dfs = dfs.append({'ID': sub,
                            'condition': condition,
                            'keyassignment': keyassignment,
                            'recallcollect': keypress_recalled_seq[i],
                            'trialcollect': trial,
                            'instructioncollect': ins_list[i],
                            'correctcollect': correctcollect}, ignore_index=True)

dfm = pd.DataFrame.from_dict(dfm)
csv_save_directory = '/Users/swu/Desktop/research/motif_learning/data/simonsays_ed/simulation_data_model_transition_recall_associative_learning_concrete_sequences.csv'
dfm.to_csv(csv_save_directory, index=False, header=True)

dfs_csv_save_directory = '/Users/swu/Desktop/research/motif_learning/data/simonsays_ed/simulation_data_model_transition_recall_associative_learning_concrete_sequences_individualkey.csv'
dfs.to_csv(dfs_csv_save_directory, index=False, header=True)

accuracy is:  0.4166666666666667
['D', 'D', 'D', 'D', 'D', 'K', 'K', 'K', 'D', 'D', 'D', 'D'] ['D', 'D', 'D', 'K', 'K', 'D', 'D', 'K', 'K', 'K', 'K', 'D']
accuracy is:  0.75
['S', 'S', 'S', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'K'] ['S', 'S', 'S', 'K', 'K', 'S', 'S', 'K', 'K', 'K', 'K', 'S']
accuracy is:  0.3333333333333333
['D', 'D', 'K', 'K', 'K', 'S', 'S', 'S', 'S', 'S', 'S', 'S'] ['D', 'D', 'D', 'K', 'K', 'D', 'D', 'K', 'K', 'K', 'K', 'D']
accuracy is:  0.25
['K', 'S', 'K', 'K', 'K', 'D', 'D', 'D', 'K', 'S', 'K', 'S'] ['K', 'K', 'K', 'S', 'S', 'K', 'K', 'S', 'S', 'S', 'S', 'K']
accuracy is:  0.25
['S', 'S', 'S', 'K', 'K', 'K', 'K', 'K', 'D', 'D', 'K', 'K'] ['S', 'S', 'S', 'J', 'J', 'S', 'S', 'J', 'J', 'J', 'J', 'S']
accuracy is:  0.4166666666666667
['K', 'K', 'K', 'K', 'S', 'S', 'S', 'K', 'K', 'K', 'K', 'K'] ['K', 'K', 'K', 'S', 'S', 'K', 'K', 'S', 'S', 'S', 'S', 'K']
accuracy is:  0.6666666666666666
['K', 'S', 'K', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D'] ['K', 'K', 'K', 'D

accuracy is:  0.5
['L', 'L', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F'] ['L', 'F', 'L', 'L', 'F', 'L', 'F', 'F', 'L', 'F', 'L', 'F']
accuracy is:  0.4166666666666667
['F', 'L', 'F', 'F', 'L', 'L', 'L', 'F', 'F', 'L', 'F', 'F'] ['F', 'F', 'L', 'F', 'L', 'F', 'L', 'L', 'F', 'F', 'L', 'L']
accuracy is:  0.6666666666666666
['L', 'F', 'L', 'F', 'F', 'L', 'F', 'L', 'L', 'L', 'L', 'L'] ['L', 'F', 'L', 'F', 'F', 'F', 'L', 'L', 'L', 'F', 'F', 'L']
accuracy is:  0.5
['F', 'L', 'F', 'F', 'F', 'L', 'L', 'L', 'F', 'F', 'F', 'F'] ['F', 'L', 'F', 'F', 'L', 'F', 'L', 'F', 'L', 'F', 'L', 'L']
accuracy is:  0.75
['L', 'F', 'F', 'F', 'F', 'L', 'L', 'L', 'F', 'L', 'F', 'F'] ['L', 'L', 'L', 'F', 'F', 'L', 'L', 'F', 'F', 'L', 'F', 'F']
accuracy is:  0.3333333333333333
['L', 'L', 'L', 'F', 'L', 'F', 'L', 'F', 'F', 'F', 'F', 'L'] ['L', 'L', 'F', 'L', 'F', 'L', 'F', 'L', 'F', 'F', 'L', 'F']
accuracy is:  0.5
['F', 'L', 'L', 'F', 'F', 'F', 'L', 'L', 'F', 'L', 'F', 'L'] ['F', 'F', 'L', 'L', 'F', 'L', 'F',

accuracy is:  0.6666666666666666
['J', 'J', 'J', 'F', 'F', 'J', 'J', 'J', 'J', 'J', 'F', 'F'] ['J', 'J', 'J', 'F', 'F', 'J', 'J', 'F', 'F', 'F', 'F', 'J']
accuracy is:  0.6666666666666666
['F', 'J', 'F', 'J', 'F', 'F', 'J', 'J', 'J', 'J', 'J', 'J'] ['F', 'F', 'F', 'J', 'J', 'F', 'F', 'J', 'J', 'J', 'J', 'F']
accuracy is:  0.5833333333333334
['J', 'J', 'J', 'F', 'F', 'F', 'F', 'F', 'F', 'J', 'J', 'F'] ['J', 'J', 'J', 'F', 'F', 'J', 'J', 'F', 'F', 'F', 'F', 'J']
accuracy is:  0.6666666666666666
['F', 'F', 'J', 'J', 'J', 'J', 'J', 'J', 'F', 'J', 'J', 'F'] ['F', 'F', 'F', 'J', 'J', 'F', 'F', 'J', 'J', 'J', 'J', 'F']
accuracy is:  0.5
['J', 'F', 'F', 'J', 'J', 'J', 'J', 'F', 'F', 'J', 'J', 'J'] ['J', 'J', 'J', 'F', 'F', 'J', 'J', 'F', 'F', 'F', 'F', 'J']
accuracy is:  0.6666666666666666
['F', 'F', 'F', 'J', 'J', 'J', 'J', 'F', 'J', 'J', 'J', 'J'] ['F', 'F', 'F', 'J', 'J', 'F', 'F', 'J', 'J', 'J', 'J', 'F']
accuracy is:  0.5833333333333334
['J', 'F', 'J', 'F', 'J', 'F', 'J', 'F', 'J', 'F', '

accuracy is:  0.3333333333333333
['F', 'F', 'F', 'F', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'F'] ['F', 'F', 'F', 'D', 'D', 'F', 'F', 'D', 'D', 'D', 'D', 'F']
accuracy is:  0.5
['K', 'K', 'K', 'K', 'K', 'K', 'S', 'S', 'K', 'K', 'K', 'K'] ['K', 'K', 'K', 'S', 'S', 'K', 'K', 'S', 'S', 'S', 'S', 'K']
accuracy is:  0.5
['S', 'S', 'S', 'K', 'K', 'K', 'K', 'S', 'S', 'S', 'S', 'S'] ['S', 'S', 'S', 'K', 'K', 'S', 'S', 'K', 'K', 'K', 'K', 'S']
accuracy is:  0.5833333333333334
['K', 'K', 'K', 'K', 'K', 'S', 'S', 'S', 'S', 'S', 'S', 'S'] ['K', 'K', 'K', 'S', 'S', 'K', 'K', 'S', 'S', 'S', 'S', 'K']
accuracy is:  0.5833333333333334
['S', 'S', 'S', 'S', 'S', 'K', 'K', 'K', 'K', 'K', 'K', 'K'] ['S', 'S', 'S', 'K', 'K', 'S', 'S', 'K', 'K', 'K', 'K', 'S']
accuracy is:  0.3333333333333333
['K', 'S', 'S', 'S', 'S', 'S', 'S', 'K', 'K', 'K', 'S', 'S'] ['K', 'K', 'K', 'S', 'S', 'K', 'K', 'S', 'S', 'S', 'S', 'K']
accuracy is:  0.5
['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'K', 'K'] ['S', 'S', 'S', 'K', 

accuracy is:  0.3333333333333333
['D', 'F', 'D', 'J', 'J', 'J', 'S', 'S', 'S', 'J', 'J', 'D'] ['D', 'D', 'J', 'D', 'J', 'D', 'J', 'D', 'D', 'J', 'J', 'J']
accuracy is:  0.25
['D', 'F', 'F', 'J', 'D', 'D', 'J', 'J', 'J', 'J', 'J', 'J'] ['D', 'D', 'F', 'D', 'D', 'F', 'D', 'F', 'D', 'F', 'F', 'F']
accuracy is:  0.25
['S', 'S', 'D', 'F', 'F', 'D', 'D', 'D', 'J', 'J', 'S', 'S'] ['S', 'J', 'S', 'J', 'S', 'J', 'J', 'S', 'S', 'J', 'J', 'S']
accuracy is:  0.4166666666666667
['J', 'D', 'J', 'J', 'J', 'J', 'J', 'J', 'S', 'S', 'S', 'J'] ['J', 'J', 'J', 'D', 'D', 'J', 'J', 'J', 'D', 'D', 'D', 'D']
accuracy is:  0.25
['S', 'S', 'J', 'J', 'J', 'J', 'J', 'F', 'S', 'J', 'J', 'F'] ['S', 'F', 'S', 'S', 'F', 'S', 'S', 'F', 'F', 'F', 'S', 'F']
accuracy is:  0.3333333333333333
['L', 'L', 'L', 'L', 'K', 'L', 'L', 'L', 'L', 'L', 'L', 'K'] ['L', 'L', 'K', 'K', 'L', 'K', 'K', 'K', 'L', 'K', 'L', 'L']
accuracy is:  0.8333333333333334
['L', 'L', 'K', 'K', 'L', 'L', 'L', 'K', 'K', 'L', 'L', 'L'] ['L', 'K', 'K', 'K

...

accuracy is:  0.5
['D', 'J', 'D', 'D', 'J', 'J', 'D', 'J', 'J', 'J', 'J', 'J'] ['D', 'J', 'J', 'J', 'J', 'J', 'D', 'D', 'D', 'D', 'J', 'D']
accuracy is:  0.4166666666666667
['J', 'D', 'J', 'J', 'J', 'D', 'D', 'D', 'D', 'D', 'D', 'J'] ['J', 'D', 'D', 'D', 'D', 'D', 'J', 'J', 'J', 'J', 'D', 'J']
accuracy is:  0.6666666666666666
['J', 'J', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'K'] ['J', 'J', 'J', 'K', 'K', 'J', 'J', 'K', 'K', 'K', 'K', 'J']
accuracy is:  0.3333333333333333
['K', 'K', 'K', 'K', 'D', 'D', 'D', 'K', 'K', 'J', 'J', 'J'] ['K', 'K', 'K', 'D', 'D', 'K', 'K', 'D', 'D', 'D', 'D', 'K']
accuracy is:  0.75
['L', 'L', 'L', 'L', 'L', 'L', 'L', 'D', 'D', 'D', 'D', 'K'] ['L', 'L', 'L', 'D', 'D', 'L', 'L', 'D', 'D', 'D', 'D', 'L']
accuracy is:  0.4166666666666667
['L', 'L', 'L', 'J', 'J', 'J', 'J', 'K', 'K', 'K', 'K', 'J'] ['L', 'L', 'L', 'J', 'J', 'L', 'L', 'J', 'J', 'J', 'J', 'L']
accuracy is:  0.25
['D', 'K', 'K', 'D', 'D', 'D', 'D', 'D', 'D', 'L', 'L', 'J'] ['D', 'D', 'D', 'K'

accuracy is:  0.5833333333333334
['S', 'S', 'S', 'F', 'F', 'F', 'S', 'F', 'F', 'F', 'F', 'S'] ['S', 'F', 'F', 'F', 'F', 'F', 'S', 'S', 'S', 'S', 'F', 'S']
accuracy is:  0.6666666666666666
['F', 'F', 'S', 'S', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F'] ['F', 'S', 'S', 'S', 'S', 'S', 'F', 'F', 'F', 'F', 'S', 'F']
accuracy is:  0.5833333333333334
['S', 'S', 'S', 'S', 'S', 'S', 'F', 'F', 'F', 'S', 'F', 'F'] ['S', 'F', 'S', 'F', 'S', 'S', 'S', 'F', 'F', 'F', 'S', 'F']
accuracy is:  0.5
['S', 'S', 'F', 'F', 'F', 'S', 'S', 'S', 'F', 'F', 'F', 'S'] ['S', 'F', 'S', 'S', 'F', 'S', 'F', 'F', 'S', 'F', 'F', 'S']
accuracy is:  0.75
['F', 'F', 'F', 'S', 'F', 'F', 'S', 'F', 'F', 'S', 'S', 'S'] ['F', 'F', 'S', 'S', 'F', 'S', 'S', 'F', 'F', 'F', 'S', 'S']
accuracy is:  0.4166666666666667
['S', 'F', 'F', 'F', 'F', 'S', 'F', 'F', 'S', 'S', 'F', 'S'] ['S', 'S', 'F', 'F', 'S', 'F', 'S', 'F', 'S', 'F', 'S', 'F']
accuracy is:  0.6666666666666666
['S', 'S', 'F', 'F', 'F', 'F', 'S', 'S', 'S', 'S', 'F', 'F'] ['S',

accuracy is:  0.6666666666666666
['L', 'L', 'L', 'L', 'F', 'F', 'L', 'L', 'L', 'L', 'L', 'L'] ['L', 'F', 'F', 'F', 'F', 'F', 'L', 'L', 'L', 'L', 'F', 'L']
accuracy is:  0.6666666666666666
['L', 'F', 'F', 'F', 'L', 'L', 'L', 'F', 'L', 'L', 'L', 'L'] ['L', 'F', 'F', 'F', 'F', 'F', 'L', 'L', 'L', 'L', 'F', 'L']
accuracy is:  0.4166666666666667
['L', 'L', 'F', 'L', 'L', 'L', 'F', 'F', 'L', 'L', 'L', 'L'] ['L', 'L', 'L', 'F', 'F', 'L', 'L', 'F', 'F', 'F', 'F', 'L']
accuracy is:  0.5833333333333334
['F', 'L', 'F', 'F', 'L', 'F', 'L', 'L', 'L', 'L', 'F', 'L'] ['F', 'F', 'F', 'L', 'L', 'F', 'F', 'L', 'L', 'L', 'L', 'F']
accuracy is:  0.5
['L', 'F', 'L', 'F', 'L', 'L', 'F', 'F', 'L', 'L', 'L', 'L'] ['L', 'L', 'L', 'F', 'F', 'L', 'L', 'F', 'F', 'F', 'F', 'L']
accuracy is:  0.5833333333333334
['F', 'F', 'L', 'L', 'L', 'F', 'L', 'L', 'F', 'F', 'L', 'L'] ['F', 'F', 'F', 'L', 'L', 'F', 'F', 'L', 'L', 'L', 'L', 'F']
accuracy is:  0.5833333333333334
['L', 'F', 'F', 'L', 'L', 'L', 'L', 'F', 'F', 'F', '

accuracy is:  0.4166666666666667
['J', 'J', 'J', 'J', 'L', 'L', 'L', 'L', 'J', 'K', 'L', 'L'] ['J', 'J', 'J', 'J', 'K', 'J', 'K', 'J', 'K', 'K', 'K', 'K']
accuracy is:  0.3333333333333333
['S', 'J', 'J', 'K', 'K', 'J', 'S', 'J', 'J', 'J', 'K', 'L'] ['S', 'S', 'J', 'S', 'S', 'J', 'J', 'S', 'J', 'S', 'J', 'J']
accuracy is:  0.4166666666666667
['J', 'S', 'J', 'K', 'L', 'L', 'J', 'J', 'J', 'K', 'K', 'J'] ['J', 'S', 'S', 'S', 'S', 'J', 'J', 'J', 'J', 'S', 'J', 'S']
accuracy is:  0.5833333333333334
['L', 'L', 'L', 'K', 'J', 'L', 'L', 'K', 'K', 'J', 'L', 'K'] ['L', 'K', 'K', 'L', 'K', 'L', 'L', 'K', 'K', 'L', 'L', 'K']
accuracy is:  0.4166666666666667
['F', 'F', 'D', 'F', 'D', 'F', 'F', 'D', 'D', 'D', 'D', 'D'] ['F', 'D', 'D', 'F', 'F', 'D', 'D', 'F', 'F', 'D', 'D', 'F']
accuracy is:  0.5833333333333334
['F', 'F', 'D', 'D', 'D', 'D', 'D', 'F', 'F', 'D', 'D', 'F'] ['F', 'F', 'F', 'D', 'F', 'D', 'D', 'D', 'D', 'F', 'D', 'F']
accuracy is:  0.5833333333333334
['F', 'D', 'D', 'D', 'D', 'D', 'F', '

accuracy is:  0.4166666666666667
['F', 'F', 'F', 'L', 'F', 'F', 'F', 'F', 'F', 'L', 'F', 'F'] ['F', 'J', 'J', 'J', 'J', 'J', 'F', 'F', 'F', 'F', 'J', 'F']
accuracy is:  0.3333333333333333
['J', 'J', 'F', 'L', 'L', 'L', 'L', 'F', 'L', 'F', 'J', 'F'] ['J', 'L', 'L', 'L', 'L', 'L', 'J', 'J', 'J', 'J', 'L', 'J']
accuracy is:  0.25
['F', 'F', 'F', 'F', 'L', 'L', 'L', 'L', 'D', 'D', 'D', 'D'] ['F', 'L', 'L', 'L', 'L', 'L', 'F', 'F', 'F', 'F', 'L', 'F']
accuracy is:  0.16666666666666666
['L', 'J', 'J', 'J', 'L', 'L', 'D', 'L', 'J', 'J', 'J', 'J'] ['L', 'D', 'D', 'D', 'D', 'D', 'L', 'L', 'L', 'L', 'D', 'L']
accuracy is:  0.25
['J', 'L', 'L', 'L', 'D', 'D', 'D', 'F', 'F', 'L', 'F', 'L'] ['J', 'D', 'D', 'D', 'D', 'D', 'J', 'J', 'J', 'J', 'D', 'J']
accuracy is:  0.16666666666666666
['D', 'D', 'D', 'F', 'F', 'J', 'J', 'J', 'J', 'J', 'D', 'D'] ['D', 'L', 'L', 'L', 'L', 'L', 'D', 'D', 'D', 'D', 'L', 'D']
accuracy is:  0.16666666666666666
['L', 'L', 'L', 'L', 'L', 'L', 'J', 'J', 'J', 'J', 'J', 'J'] [

(array([[[6.]],
 
        [[2.]],
 
        [[2.]],
 
        [[1.]],
 
        [[1.]],
 
        [[1.]],
 
        [[1.]],
 
        [[1.]],
 
        [[1.]],
 
        [[1.]],
 
        [[1.]],
 
        [[1.]]]),
 86028    S
 86029    D
 86030    D
 86031    D
 86032    D
 86033    D
 86034    S
 86035    S
 86036    S
 86037    S
 86038    D
 86039    S
 Name: instructioncollect, dtype: object)